In [40]:
import os
from firebase_admin import auth, credentials, firestore, initialize_app
from dotenv import load_dotenv
import re

In [30]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Access variables
firebase_key_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
project_id = os.getenv("FIRESTORE_PROJECT_ID")
database_url = os.getenv("FIRESTORE_DATABASE_URL")

print("Firebase Key Path:", firebase_key_path)
print("Firestore Project ID:", project_id)
print("Firestore Database URL:", database_url)


Firebase Key Path: .streamlit/tournalytics-firebase-adminsdk-dbcy8-e1435aaaa6.json
Firestore Project ID: tournalytics
Firestore Database URL: https://firestore.googleapis.com/v1/projects/tournalytics/databases/(default)/documents


In [31]:
# Initialize Firebase Admin SDK
try:
    if not firebase_admin._apps:
        cred = credentials.Certificate(firebase_key_path)
        firebase_admin.initialize_app(cred, {"projectId": project_id})
    print("Firebase initialized successfully!")
except Exception as e:
    print("Error initializing Firebase:", e)


Firebase initialized successfully!


In [32]:
# Test Firestore connection
try:
    db = firestore.client()
    users_ref = db.collection("users")
    docs = users_ref.stream()
    
    print("Users collection documents:")
    for doc in docs:
        print(doc.id, doc.to_dict())
except Exception as e:
    print("Error connecting to Firestore:", e)

Users collection documents:
dmf95 {'created_at': DatetimeWithNanoseconds(2025, 1, 6, 15, 55, 10, 319919, tzinfo=datetime.timezone.utc), 'email': 'domenicfayad@gmail.com', 'league_id': None, 'username': 'dmf95', 'role': 'super_admin'}


In [26]:
from firebase_admin import firestore
from datetime import datetime

# Initialize Firestore client
db = firestore.client()

# Define metadata for the user
email = "domenicfayad@gmail.com"
username = "dmf95"  # Extracted or defined username
user_metadata = {
    "username": username,
    "email": email,
    "role": "super_admin",  # Replace with appropriate role ('user', 'admin', 'super_admin')
    "league_id": None,  # Add associated league if applicable
    "created_at": datetime.now(),
}

# Save the metadata in the Firestore `users` collection
try:
    db.collection("users").document(username).set(user_metadata)
    print(f"User metadata for '{username}' saved in Firestore.")
except Exception as e:
    print("Error saving user metadata:", e)


User metadata for 'dmf95' saved in Firestore.


In [45]:
def create_user_metadata(email, password, username, role="user", league_id=None):
    """
    Create a new user in Firebase Authentication and store additional metadata in Firestore.
    """
    try:
        # Create user in Firebase Authentication
        user_record = auth.create_user(email=email, password=password, display_name=username)

        # Store additional metadata in Firestore
        user_doc = {
            "uid": user_record.uid,  # Link Firestore metadata to Firebase Auth UID
            "username": username,
            "email": email,
            "role": role,
            "league_id": league_id,
            "created_at": firestore.SERVER_TIMESTAMP,
        }
        db.collection("users").document(user_record.uid).set(user_doc)

        print(f"User {username} created successfully.")
        return True
    except Exception as e:
        print(f"Error creating user metadata: {e}")
        return False

# Test creating a new user
create_user_metadata(
    email="domenicfayad@gmail.com",
    password="elephant",
    username="dmf95",
    role="super_admin"
)


Error creating user metadata: The user with the provided email already exists (EMAIL_EXISTS).


False

In [49]:
def authenticate_user(identifier, password=None):
    """
    Authenticate a user by either email or username.

    Password verification is handled via Firebase Authentication.
    """
    try:
        # Determine if the identifier is an email
        is_email = re.match(r"[^@]+@[^@]+\.[^@]+", identifier)

        if is_email:
            # Authenticate by email
            firebase_user = auth.get_user_by_email(identifier)
            uid = firebase_user.uid
        else:
            # Authenticate by username
            user_ref = db.collection("users").where("username", "==", identifier).limit(1).get()
            if not user_ref:
                print("Username not found in Firestore.")
                return None
            user_data = user_ref[0].to_dict()
            uid = user_ref[0].id  # Firestore document ID corresponds to UID
            identifier = user_data.get("email")  # Retrieve the associated email for Firebase Authentication
            
            # Fetch Firebase user by email
            try:
                firebase_user = auth.get_user_by_email(identifier)
            except firebase_admin.auth.UserNotFoundError:
                print("Email associated with username not found in Firebase Authentication.")
                return None

        # Verify the password using Firebase Authentication
        # Password verification happens on the client-side or via a token exchange in Firebase SDK.
        if not firebase_user:
            print("Firebase Authentication failed.")
            return None

        # Fetch user metadata from Firestore
        user_metadata = db.collection("users").document(uid).get().to_dict()
        if not user_metadata:
            print("User metadata not found in Firestore.")
            return None

        # Return user metadata
        return {
            "username": user_metadata.get("username"),
            "role": user_metadata.get("role"),
            "email": user_metadata.get("email"),
            "league_id": user_metadata.get("league_id"),
        }
    except Exception as e:
        print(f"Error authenticating user: {e}")
        return None


# Test authenticating a user
authenticated_user = authenticate_user(identifier="domenicfayad@gmail.com", password="elephant")
print(authenticated_user)


{'username': 'dmf95', 'role': 'super_admin', 'email': 'domenicfayad@gmail.com', 'league_id': None}


In [61]:
import json

# Firestore client
db = firestore.client()

# Load dictionary from file
with open("assets/tournament_ba6b4c4e_050120251954.json", "r") as f:
    tournament_data = json.load(f)

# Specify Firestore path
doc_path = f"tournaments/{tournament_data}"

# Upload to Firestore
#db.document(doc_path).set(tournament_data)
#print(f"Tournament data uploaded to Firestore at {doc_path}")


In [63]:
import json
from datetime import datetime

def save_tournament_to_file(tournament_data):
    """
    Combine dictionaries and save to a JSON file with a unique filename.
    
    :param tournament_data: Combined dict
    :return: The generated filename.
    """
    # Ensure tournament_id exists in metadata
    tournament_id = tournament_data['metadata'].get("tournament_id")
    if not tournament_id:
        raise ValueError("The 'tournament_id' must be present in the metadata.")

    # Generate a unique filename using tournament_id and current timestamp
    timestamp = datetime.now().strftime("%d%m%Y%H%M")
    filename = f"tournament_{tournament_id}_{timestamp}.json"

    # Save to file
    try:
        with open(filename, "w") as f:
            json.dump(tournament_data, f, indent=4)
        print(f"Tournament data successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving tournament data: {e}")
        raise

    return filename


In [62]:
tournament_data['metadata']

{'tournament_id': 'ba6b4c4e',
 'tournament_name': '2025 Cam Brock Invitational',
 'event_date': '2025-01-01',
 'tournament_type': 'Round Robin',
 'num_players': 6,
 'num_consoles': 2,
 'half_duration': 5,
 'selected_players': ['Domenic Fayad',
  'Cameron Brock',
  'Thomas Tassone',
  'Michael Hanuman',
  'Luigi Polisi',
  'Aria Sarhangpour'],
 'team_selection': {'Domenic Fayad': 'Manchester United',
  'Cameron Brock': 'Liverpool',
  'Thomas Tassone': 'Chelsea',
  'Michael Hanuman': 'Manchester City',
  'Luigi Polisi': 'Real Madrid',
  'Aria Sarhangpour': 'Barcelona'}}